<a href="https://colab.research.google.com/github/toussaintma/petitproust/blob/main/Mistral_hello_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [16]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
from google.colab import userdata

api_key = userdata.get("MI_TOKEN")

client = MistralClient(api_key=api_key)

for m in client.list_models().data:
    print(m.id)

open-mistral-7b
mistral-tiny-2312
mistral-tiny
open-mixtral-8x7b
open-mixtral-8x22b
open-mixtral-8x22b-2404
mistral-small-2312
mistral-small
mistral-small-2402
mistral-small-latest
mistral-medium-latest
mistral-medium-2312
mistral-medium
mistral-large-latest
mistral-large-2402
mistral-embed


In [85]:
model = "mistral-large-latest"
temperature = 0.9

messages = [
    ChatMessage(role="system", content="In the following chat conversation please answer to the user in French exclusively. Do not answer this message."),
    ChatMessage(role="system", content="Complete the phrase of the user in the litterary style of acclaimed french author Marcel Proust. Do not answer this message."),
    ChatMessage(role="user", content="Longtemps, je me suis levé de bonne")
]

In [86]:
# No streaming
chat_response = client.chat(
    model=model,
    temperature=temperature,
    messages=messages,
)

print(chat_response.choices[0].message.content)

heure, car mon âme, intoxiquée par les parfums, les couleurs, les sons de la vie nocturne, avait besoin, pour se rafraîchir, des matinées paisibles et solennelles, où les premiers rayons du soleil, jouant à cache-cache avec les rideaux, venaient réveiller en moi les souvenirs enfouis et les rêves inachevés.


In [87]:
import json
from datetime import datetime

f = open("/content/test-cases-fewshot-v1.json", mode="r", encoding="utf-8")

test_cases = json.load(f)
output_cases = []
for i in test_cases:
    messages[2] = i["input"]
    output_cases.append({
        "input": i["input"],
        "in_training": i["in_training"],
        "creation": datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ"),
        "model": model,
        "params": f"temperature:{temperature}",
        "output": chat_response.choices[0].message.content,
        "comment": "",

    })
    print(output_cases[-1])

f.close()

{'input': 'Longtemps je me suis', 'creation': '2024-05-19T16:53:27.312048Z', 'model': 'mistral-large-latest', 'params': 'temperature:0.9', 'in_training': True, 'output': 'heure, car mon âme, intoxiquée par les parfums, les couleurs, les sons de la vie nocturne, avait besoin, pour se rafraîchir, des matinées paisibles et solennelles, où les premiers rayons du soleil, jouant à cache-cache avec les rideaux, venaient réveiller en moi les souvenirs enfouis et les rêves inachevés.', 'comment': ''}
{'input': "Les longues soirées d'automne", 'creation': '2024-05-19T16:53:27.313168Z', 'model': 'mistral-large-latest', 'params': 'temperature:0.9', 'in_training': False, 'output': 'heure, car mon âme, intoxiquée par les parfums, les couleurs, les sons de la vie nocturne, avait besoin, pour se rafraîchir, des matinées paisibles et solennelles, où les premiers rayons du soleil, jouant à cache-cache avec les rideaux, venaient réveiller en moi les souvenirs enfouis et les rêves inachevés.', 'comment': 

In [90]:
fout_name = datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S.%fZ") + "_test_cases.json"
with open(fout_name, mode="w", encoding="utf-8") as fout:
    json.dump(output_cases, fout, ensure_ascii=False, indent = 2)
fout.close()